## 使用字符RNN生成莎士比亚文本

在2015年的一篇著名博客文章 "https://karpathy.github.io/2015/05/21/rnn-effectiveness/" 中，Andrej Karpathy展示了如何训练循环神经网络来预测句子中的下一个字符。这个char-RNN可以用来生成小说文本，每次一个字符。

In [22]:
# tf.keras.utils.get_file()函数下载莎士比亚的所有作品
import tensorflow as tf

shakespeare_url = "https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt"
filepath = tf.keras.utils.get_file("shakespeare.txt", shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [4]:
print(shakespeare_text[:80])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.


In [2]:
# 使用tf.keras.layers.TextVectorization层对此文本进行编码。我们将设置split="character"，基于字符进行编码，而不是默认的基于单词进行编码，并使用standardize="lower"将文本转换为小写（这将简化任务）：
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([shakespeare_text])


encoded = text_vec_layer([shakespeare_text])[0]
encoded


<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([21,  7, 10, ..., 22, 28, 12], dtype=int64)>

In [3]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 ' ',
 'e',
 't',
 'o',
 'a',
 'i',
 'h',
 's',
 'r',
 'n',
 '\n',
 'l',
 'd',
 'u',
 'm',
 'y',
 'w',
 ',',
 'c',
 'f',
 'g',
 'b',
 'p',
 ':',
 'k',
 'v',
 '.',
 "'",
 ';',
 '?',
 '!',
 '-',
 'j',
 'q',
 'x',
 'z',
 '3',
 '&',
 '$']

In [4]:
# 每个字符现在都映射到一个整数，从2开始。TextVectorization层将值0保留为填充词元(token)，并将值1保留为未知字符。
# 现在不需要这两个词元，因此从字符ID数中减去2，并计算不同字符的总数和总字符数：
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)

print(n_tokens)

39


将这个非常长的序列转换成一个窗口数据集，然后使用它来训练一个序列到序列的循环神经网络。目标序列将类似于输入序列，但是会向“未来”移动一个时间步。例如，数据集中的样本可能是一个由表示文本“to be or not to b”（不包括最后一个“e”）的字符ID组成的序列，相应的目标序列是一个由表示文本“o be or not to be”（包括最后一个“e”，但不包括开头的“t”）的字符ID组成的序列

In [5]:
# 将字符ID序列转换成输入/目标窗口对的数据集
def to_dataset(sequence, length, shuffle=False, seed=None, batch_size=32):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=1, drop_remainder=True)

    ds = ds.flat_map(lambda window_ds: window_ds.batch(length + 1))


    if shuffle:
        ds = ds.shuffle(buffer_size=100_000, seed=seed)
    ds = ds.batch(batch_size)
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

- 它将序列作为输入（即编码文本），并创建一个包含所需长度的所有窗口的数据集。
- 它将长度增加1，因为我们需要将下一个字符放在目标序列中。
- 然后，它对窗口进行乱序处理（可选），将它们分批，将它们拆分为输入/输出对，并激活预取功能。

长度为11的窗口和批量大小3。每个窗口的起始索引都显示在旁边：

![准备乱序窗口的数据集](./images/RNN/p7.png)

In [6]:
# 大约90%的文本进行训练，5%用于验证，5%用于测试
length = 100  # length决定循环神经网络能学习的最长模式
tf.random.set_seed(42)

train_set = to_dataset(encoded[:1_000_000], length=length, shuffle=True, seed=42)
valid_set = to_dataset(encoded[1_000_000: 1_060_000], length=length)
test_set = to_dataset(encoded[1_060_000:], length=length)

In [7]:
for data in valid_set.take(1):
    print(data)

(<tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 5,  7,  0, ...,  6,  1,  0],
       [ 7,  0,  8, ...,  1,  0, 18],
       [ 0,  8,  1, ...,  0, 18,  6],
       ...,
       [ 4,  2,  0, ...,  3, 26, 10],
       [ 2,  0,  7, ..., 26, 10, 10],
       [ 0,  7,  6, ..., 10, 10,  2]], dtype=int64)>, <tf.Tensor: shape=(32, 100), dtype=int64, numpy=
array([[ 7,  0,  8, ...,  1,  0, 18],
       [ 0,  8,  1, ...,  0, 18,  6],
       [ 8,  1,  4, ..., 18,  6,  3],
       ...,
       [ 2,  0,  7, ..., 26, 10, 10],
       [ 0,  7,  6, ..., 10, 10,  2],
       [ 7,  6,  1, ..., 10,  2,  8]], dtype=int64)>)


### 构建和训练char-RNN模型

- 使用Embedding层作为第一层，以编码字符ID。Embedding层的输入维数是不同字符ID的数量，输出维数是可以调整的超参数——现在将其设置为16。Embedding层的输入将是形状为［批量大小，窗口长度］的二维张量，Embedding层的输出将是形状为［批量大小，窗口长度，嵌入大小］的三维张量。
- 为输出层使用Dense层：它必须具有39个单元(n_tokens)，因为文本中有39个不同的字符，希望在每个时间步输出每个可能字符的概率。每个时间步39个输出概率的总和应该为1，因此对Dense层的输出应用softmax激活函数。
- 最后，使用"sparse_categorical_crossentropy"损失和Nadam优化器编译此模型，并使用ModelCheckpoint回调函数在训练过程中保存最佳模型（以验证精度为标准）进行多个轮次的训练。

In [ ]:
# 不用简单RNN,用带有长短记忆的GRU
#
# [[ 5,  7,  0, ...,  6,  1,  0]]  1 * 100

#   embedding
# [[ v5  v7  v0  .... v6  v1  v0]] 1 * 100 * 16

# n_tokens:39
# embedding： 39 * n 列的矩阵：  5行：  [w0, w1, ... wn-1]

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True
)
history = model.fit(train_set, validation_data=valid_set, epochs=10, callbacks=[model_ckpt])

Epoch 1/10
  10781/Unknown - 640s 58ms/step - loss: 1.4954 - accuracy: 0.5484

In [20]:
# 训练后搭建最终模型
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X-2), # 不使用填充词元(0)  和 未知词元 (1)
    model
])

In [23]:
y_proba = shakespeare_model.predict(["To be or not to be"])[0, -1]
y_proba
y_pred = tf.argmax(y_proba)  # 选择概率最高的字母ID

print(text_vec_layer.get_vocabulary())

text_vec_layer.get_vocabulary()[y_pred + 2]

1/1 [==============================] - 0s 41ms/step
['', '[UNK]', ' ', 'e', 't', 'o', 'a', 'i', 'h', 's', 'r', 'n', '\n', 'l', 'd', 'u', 'm', 'y', 'w', ',', 'c', 'f', 'g', 'b', 'p', ':', 'k', 'v', '.', "'", ';', '?', '!', '-', 'j', 'q', 'x', 'z', '3', '&', '$']


'l'

### 生成莎士比亚文本

要使用char-RNN模型生成新文本，可以将一些文本输入模型，让模型预测最有可能的下一个字母，将其添加到文本的末尾，然后将扩展后的文本输入模型来猜测下一个字母，以此类推。这叫作贪婪解码。但是在实践中，这往往导致相同的单词一遍又一遍地重复。

相反，可以使用TensorFlow的tf.random.categorical()函数随机采样下一个字符，采样概率等于估计概率。这将生成更多样化和有趣的文本。categorical()函数在给定类别对数概率(logits)的情况下，对随机类别指数进行采样。

In [1]:
import tensorflow as tf
log_probas = tf.math.log([[0.5, 0.4, 0.1]])
tf.random.set_seed(42)

n = 2000
tf.random.categorical(log_probas, num_samples=n)



<tf.Tensor: shape=(1, 2000), dtype=int64, numpy=array([[0, 1, 0, ..., 1, 0, 2]], dtype=int64)>

为了更好地控制生成文本的多样性，可以将logits（对数概率）除以一个称为“温度”的数字，这个数字可以根据需求进行调整。“温度”接近零将更偏向于高概率字符，而较高“温度”则会使所有字符获得相同的概率。通常在生成相对严谨和精确的文本（例如数学公式）时，较低的“温度”更为适用，而在生成更多样化且有创意的文本时，则适合用较高的“温度”。

In [39]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [8]:
def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [41]:
tf.random.set_seed(42)
print(extend_text("To be or not to be", temperature=10))


1/1 [==============================] - 0s 34ms/step
To be or not to bedf3zwvcik :ua!&q. :phgr&;ubltcpzhp:'rv:cq3z!$ pau:


In [28]:
print(extend_text("To be or not to be", temperature=1))

1/1 [==============================] - 0s 16ms/step
To be or not to be good and wrong; but come. though which the fine.



In [29]:
print(extend_text("To be or not to be", temperature=100))

1/1 [==============================] - 0s 16ms/step
To be or not to bef ,mt'&ozfpady-$
wh!nse?pws3ert--vgerdjw?c-y-ewxnj


### 模拟生成莎士比亚文本的流程 生成名字

In [2]:
with open("datasets/dino.txt", "r") as f:
    dino_names = f.read()
    dino_names = dino_names.lower()

In [3]:
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")

text_vec_layer.adapt([dino_names])
encoded = text_vec_layer([dino_names])[0]
encoded

<tf.Tensor: shape=(19909,), dtype=int64, numpy=array([ 2,  2, 15, ...,  4,  4, 12], dtype=int64)>

In [4]:
text_vec_layer.get_vocabulary()

['',
 '[UNK]',
 'a',
 's',
 'u',
 'o',
 'r',
 '\n',
 'n',
 'i',
 'e',
 't',
 'l',
 'p',
 'h',
 'c',
 'g',
 'd',
 'm',
 'y',
 'b',
 'k',
 'v',
 'x',
 'z',
 'j',
 'w',
 'f',
 'q']

In [5]:
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
dataset_size = len(encoded)
n_tokens

27

In [6]:
end_of_name_encode = text_vec_layer(["\n"])[0,0].numpy() - 2
end_of_name_encode

5

In [9]:
# 把 Tensor 转成 numpy 数组
encoded_np = encoded.numpy()

# 存放 (X, Y) 对
names_X = []  # [第一个名字不包括换行符， 第二个名字不包括换行符， ...  最后一个名字不包括换行符]
names_Y = []  # [第一个名字有换行符，但无第一个字符， .....    , ...  最后一个名字有换行符，但无第一个字符]

# 临时缓冲区
current_name = []

for token in encoded_np:
    current_name.append(token)
    if token == end_of_name_encode:
        if len(current_name) > 1:  # 至少两个字符才能形成 X/Y
            X = current_name[:-1]
            Y = current_name[1:]
            names_X.append(X)
            names_Y.append(Y)
        current_name = []


current_name.append(end_of_name_encode)
if len(current_name) > 1:
    X = current_name[:-1]
    Y = current_name[1:]
    names_X.append(X)
    names_Y.append(Y)
    current_name = []

names_X[:5], names_Y[:5]
# # 转成 TensorFlow 张量
X_dataset = tf.ragged.constant(names_X)
X_dataset
Y_dataset = tf.ragged.constant(names_Y)
# Y_dataset
#
train_set = tf.data.Dataset.from_tensor_slices((X_dataset, Y_dataset)).shuffle(1000).batch(8)
# print("共提取名字数:", len(names_X))


(<tf.RaggedTensor [[13, 12, 2, 0, 6, 15, 3, 6, 14, 3, 13, 3, 8, 10, 2, 4, 2, 1],
 [18, 0, 4, 4, 3, 1, 0, 1, 0, 2, 4, 2, 1],
 [16, 0, 6, 3, 1, 11, 3, 6, 15, 17, 10, 2, 1],
 [18, 4, 0, 15, 17, 13, 6, 8, 16, 8], [15, 0, 21, 7, 0, 9, 7, 9, 0, 6],
 [0, 10, 18, 8, 4, 9, 0, 15, 4, 3, 16, 8, 2, 1],
 [0, 4, 13, 12, 0, 8, 3, 13, 8, 4, 0, 9, 3, 11, 1],
 [18, 0, 7, 6, 3, 13, 8, 4, 0, 9, 3, 11, 1]]>, <tf.RaggedTensor [[12, 2, 0, 6, 15, 3, 6, 14, 3, 13, 3, 8, 10, 2, 4, 2, 1, 5],
 [0, 4, 4, 3, 1, 0, 1, 0, 2, 4, 2, 1, 5],
 [0, 6, 3, 1, 11, 3, 6, 15, 17, 10, 2, 1, 5],
 [4, 0, 15, 17, 13, 6, 8, 16, 8, 5], [0, 21, 7, 0, 9, 7, 9, 0, 6, 5],
 [10, 18, 8, 4, 9, 0, 15, 4, 3, 16, 8, 2, 1, 5],
 [4, 13, 12, 0, 8, 3, 13, 8, 4, 0, 9, 3, 11, 1, 5],
 [0, 7, 6, 3, 13, 8, 4, 0, 9, 3, 11, 1, 5]]>)


In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(32, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

history = model.fit(X_dataset, Y_dataset, batch_size=1, epochs=10)
model.fit(train_set, epochs=20)

Epoch 1/20
192/192 [==============================] - 1s 7ms/step - loss: 1.5511 - accuracy: 0.5330
Epoch 2/20
192/192 [==============================] - 1s 7ms/step - loss: 1.5414 - accuracy: 0.5365
Epoch 3/20
192/192 [==============================] - 1s 6ms/step - loss: 1.5345 - accuracy: 0.5379
Epoch 4/20
192/192 [==============================] - 1s 6ms/step - loss: 1.5246 - accuracy: 0.5409
Epoch 5/20
192/192 [==============================] - 1s 6ms/step - loss: 1.5169 - accuracy: 0.5440
Epoch 6/20
192/192 [==============================] - 1s 6ms/step - loss: 1.5082 - accuracy: 0.5468
Epoch 7/20
192/192 [==============================] - 1s 6ms/step - loss: 1.5030 - accuracy: 0.5482
Epoch 8/20
192/192 [==============================] - 1s 6ms/step - loss: 1.4946 - accuracy: 0.5525
Epoch 9/20
192/192 [==============================] - 1s 6ms/step - loss: 1.4901 - accuracy: 0.5535
Epoch 10/20
192/192 [==============================] - 1s 6ms/step - loss: 1.4817 - accuracy: 0.5547

In [13]:
dino_name_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X -2),
    model
])

In [14]:
def my_next_char(model, text, temperature=1):
    y_proba = model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

In [17]:
def my_extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        char_gen_next = my_next_char(dino_name_model, text, temperature)
        if char_gen_next == "\n":
            break
        text += my_next_char(dino_name_model, text, temperature)
    return text

In [19]:
my_extend_text("aa")

1/1 [==============================] - 0s 34ms/step


'aaroptorolopheyis'

## 有状态的RNN

刚才的模型无法学习长度超100个字符的模式，使用有状态的RNN可以学习更长序列。

到目前为止，只使用了无状态RNN：在每次训练迭代中，模型从一个全是零的隐藏状态开始，然后在每个时间步更新这个状态，在最后一个时间步之后，将其丢弃，因为不再需要它了。

如果指示RNN在处理完一个训练批次后保留该最终状态，并将其用作下一个训练批次的初始状态，那么模型可以学习长期模式，尽管只通过短序列进行反向传播。这就是所谓的有状态RNN。

首先需要注意的是，只有在批次中的每个输入序列都从相应的上一批次序列结束的位置开始时才能建立有状态RNN。因此，构建有状态RNN时，首先需要使用非重叠顺序输入序列（而不是用于训练无状态RNN的随机重叠序列）。当创建tf.data.Dataset时，需要在调用window()方法时使用shift=length（而不是shift=1）。此外，不能调用shuffle()方法。

在为有状态RNN准备数据集时，批处理要比无状态RNN时更困难。如果调用batch(32)，那么32个连续的窗口将被放入同一个批次中，接下来的批次将无法继续从这些窗口的最后一个位置开始。第一个批次将包含窗口1到32，第二个批次将包含窗口33到64，因此如果考虑每个批次的第一个窗口（即窗口1和33），便会发现它们不是连续的。这个问题最简单的解决方案就是只使用批量大小1。下面的to_dataset_for_stateful_rnn()自定义实用函数使用这种策略来为有状态RNN准备数据集：

![为有状态RNN准备连续序列片段的数据集](./images/RNN/p10.png)

In [20]:
def to_dataset_for_stateful_rnn(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length+1, shift=length, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(length+1)).batch(1)

    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

In [23]:
import tensorflow as tf
text_vec_layer = tf.keras.layers.TextVectorization(split="character", standardize="lower")
text_vec_layer.adapt([shakespeare_text])
encoded = text_vec_layer([shakespeare_text])[0]
encoded -= 2
n_tokens = text_vec_layer.vocabulary_size() - 2
length = 100  # length决定循环神经网络能学习的最长模式

In [24]:
stateful_train_set = to_dataset_for_stateful_rnn(encoded[:1_000_000], length)
stateful_valid_set = to_dataset_for_stateful_rnn(encoded[1_000_000:1_060_000], length)
stateful_test_set = to_dataset_for_stateful_rnn(encoded[1_060_000:], length)

现在创建有状态RNN。当创建每个循环层时，需要将stateful参数设置True，因为有状态RNN需要知道批量大小（因为它将为批处理中的每个输入序列保留状态）。因此，必须在第一层设置batch_input_shape参数。请注意，可以不指定第二维度，因为输入序列可以具有任意长度

In [25]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16, batch_input_shape=[1, None]),
    tf.keras.layers.GRU(128, return_sequences=True, stateful=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])

In [26]:
# 每个轮次结束时，需要在返回到文本开头之前重置状态
class ResetStatesCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [7]:
model_ckpt = tf.keras.callbacks.ModelCheckpoint(
    "my_shakespeare_model", monitor="val_accuracy", save_best_only=True
)
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(stateful_train_set, validation_data=stateful_valid_set, epochs=10, callbacks=[ResetStatesCallback(), model_ckpt])

Epoch 1/10
   9996/Unknown - 144s 14ms/step - loss: 1.8666 - accuracy: 0.4508INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 147s 15ms/step - loss: 1.8665 - accuracy: 0.4508 - val_loss: 1.7065 - val_accuracy: 0.4911
Epoch 2/10
9995/9999 [============================>.] - ETA: 0s - loss: 1.5619 - accuracy: 0.5286INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 139s 14ms/step - loss: 1.5618 - accuracy: 0.5287 - val_loss: 1.6223 - val_accuracy: 0.5136
Epoch 3/10
9996/9999 [============================>.] - ETA: 0s - loss: 1.4829 - accuracy: 0.5491INFO:tensorflow:Assets written to: my_shakespeare_model\assets


...

9999/9999 [==============================] - 142s 14ms/step - loss: 1.4037 - accuracy: 0.5694 - val_loss: 1.5435 - val_accuracy: 0.5397
Epoch 7/10
9999/9999 [==============================] - ETA: 0s - loss: 1.3918 - accuracy: 0.5723INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 144s 14ms/step - loss: 1.3918 - accuracy: 0.5723 - val_loss: 1.5354 - val_accuracy: 0.5413
Epoch 8/10
9997/9999 [============================>.] - ETA: 0s - loss: 1.3825 - accuracy: 0.5746INFO:tensorflow:Assets written to: my_shakespeare_model\assets


INFO:tensorflow:Assets written to: my_shakespeare_model\assets


9999/9999 [==============================] - 144s 14ms/step - loss: 1.3825 - accuracy: 0.5746 - val_loss: 1.5321 - val_accuracy: 0.5439
Epoch 9/10
9999/9999 [==============================] - 142s 14ms/step - loss: 1.3754 - accuracy: 0.5766 - val_loss: 1.5354 - val_accuracy: 0.5434
Epoch 10/10
9999/9999 [==============================] - 141s 14ms/step - loss: 1.3704 - accuracy: 0.5778 - val_loss: 1.5290 - val_accuracy: 0.5436


在训练后，只能使用与训练期间相同大小的批次进行预测。为避免此限制，请创建相同的无状态模型，并将有状态模型的权重复制到该模型

有趣的是，尽管char-RNN模型只是被训练来预测下一个字符，但是这个看似简单的任务实际上要求它学习一些更高级的任务。例如，查找“Great movie，I really”的下一个字符时，了解该句子是正面的是有帮助的，因此接下来的字符可能是“l”［表示“喜欢”(loved)］而不是“h”［表示“讨厌”(hated)］

OpenAI在一篇论文中描述了他们如何在大型数据集上训练大型char-RNN模型，并发现其中的一个神经元作为出色的情感分析分类器：尽管该模型在没有任何标签的情况下进行了训练，但“情感神经元”在情感分析基准测试中达到了最先进的性能。这预示并激发了NLP中无监督预训练的应用。

虽然批处理更难，但也不是不可能的。例如，可以将莎士比亚的文本分成32个长度相等的文本，为每个文本创建一个连续的输入序列数据集，最后使用tf.data.Dataset.zip(datasets).map(lambda*windows：tf.stack(windows))创建适当的连续批次，批次中的第n个输入序列恰好从上一个批次中第n个输入序列结束的地方开始

In [9]:
def next_char(text, temperature=1):
    y_proba = shakespeare_model.predict([text])[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0,0]
    return text_vec_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [10]:
stateless_model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=n_tokens, output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(n_tokens, activation="softmax")
])


stateless_model.build(tf.TensorShape([None, None]))
stateless_model.set_weights(model.get_weights())
shakespeare_model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Lambda(lambda X: X - 2),  # no <PAD> or <UNK> tokens
    stateless_model
])

tf.random.set_seed(42)
print(extend_text("to be or not to be", temperature=0.01))

1/1 [==============================] - 0s 16ms/step
to be or not to be a shall be a shall be a shall be a shall be a sha


In [42]:
import numpy as np
seq = tf.range(20)
parts = np.array_split(seq, 2)
datasets = tuple(to_non_overlapping_windows(part, 3) for part in parts)

In [50]:
list(tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows)))

[<tf.Tensor: shape=(2, 4), dtype=int32, numpy=
 array([[ 0,  1,  2,  3],
        [10, 11, 12, 13]])>,
 <tf.Tensor: shape=(2, 4), dtype=int32, numpy=
 array([[ 3,  4,  5,  6],
        [13, 14, 15, 16]])>,
 <tf.Tensor: shape=(2, 4), dtype=int32, numpy=
 array([[ 6,  7,  8,  9],
        [16, 17, 18, 19]])>]

In [51]:
# extra code – shows one way to prepare a batched dataset for a stateful RNN

import numpy as np

def to_non_overlapping_windows(sequence, length):
    ds = tf.data.Dataset.from_tensor_slices(sequence)
    ds = ds.window(length + 1, shift=length, drop_remainder=True)
    return ds.flat_map(lambda window: window.batch(length + 1))

def to_batched_dataset_for_stateful_rnn(sequence, length, batch_size=32):
    parts = np.array_split(sequence, batch_size)
    datasets = tuple(to_non_overlapping_windows(part, length) for part in parts)
    ds = tf.data.Dataset.zip(datasets).map(lambda *windows: tf.stack(windows))
    return ds.map(lambda window: (window[:, :-1], window[:, 1:])).prefetch(1)

list(to_batched_dataset_for_stateful_rnn(tf.range(20), length=3, batch_size=2))

[(<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 0,  1,  2],
         [10, 11, 12]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 1,  2,  3],
         [11, 12, 13]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 3,  4,  5],
         [13, 14, 15]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 4,  5,  6],
         [14, 15, 16]])>),
 (<tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 6,  7,  8],
         [16, 17, 18]])>,
  <tf.Tensor: shape=(2, 3), dtype=int32, numpy=
  array([[ 7,  8,  9],
         [17, 18, 19]])>)]

## 情感分析

生成文本的任务既有趣又有教学意义，但在真实项目中，NLP最常用的应用之一是文本分类——特别是情感分析。如果MNIST数据集上的图像分类是计算机视觉的“Hello World！”，那么IMDb电影评论数据集上的情感分析就是自然语言处理的“HelloWorld！”。IMDb数据集包括50000条英文电影评论（25000条用于训练，25000条用于测试），这些评论是从著名的互联网电影数据库 (https://imdb.com) 中提取的，每条评论还有一个简单的二元目标值，用以指示每条评论是负面的(0)还是正面的(1)。就像MNIST一样，IMDb电影评论数据集受欢迎是有原因的：它足够简单，可以在合理的时间内在CPU上处理。

In [52]:
import tensorflow_datasets as tfds
import tensorflow as tf
raw_train_set, raw_valid_set, raw_test_set = tfds.load(
    name="imdb_reviews",
    split=["train[:90%]", "train[90%:]", "test"],
    as_supervised=True
)
tf.random.set_seed(42)
train_set = raw_train_set.shuffle(5000, seed=42).batch(32).prefetch(1)
valid_set = raw_valid_set.batch(32).prefetch(1)
test_set = raw_test_set.batch(32).prefetch(1)

In [53]:
for review, label in raw_train_set.take(4):
    print(review.numpy().decode("utf-8")[:200], "...")
    print("Label:", label.numpy())

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting  ...
Label: 0
I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However  ...
Label: 0
Mann photographs the Alberta Rocky Mountains in a superb fashion, and Jimmy Stewart and Walter Brennan give enjoyable performances as they always seem to do. <br /><br />But come on Hollywood - a Moun ...
Label: 0
This is the kind of film for a snowy Sunday afternoon when the rest of the world can go ahead with its own business as you descend into a big arm-chair and mellow for a couple of hours. Wonderful perf ...
Label: 1


 为了建立这个任务的模型，我们需要对文本进行预处理，但这一次我们将把文本拆分成单词而不是字符。为此，我们再次使用tf.keras.layers.TextVectorization层。请注意，它使用空格来确定单词边界，这在某些语言中不太合适。例如，汉语写作单词之间不使用空格，即使在英语中，空格也并不总是分词的最佳方式

幸运的是，有解决这些问题的方法。在2016年的一篇论文中探讨了几种子词级别的文本分词和重组方法。这样，即使模型遇到了它以前从未见过的生僻词，它仍然可以合理地猜测它的含义。例如，即使模型在训练期间从未见过单词smartest，如果它学到了单词smart并且还学到了后缀est的意思是“最”，它仍然可以推断出smartest的含义。作者评估的技术之一是字节对编码(Byte Pair Encoding，BPE)。BPE的工作原理是将整个训练集拆分为单个字符（包括空格），然后反复合并最常见的相邻字符对，直到词汇达到所需的大小。（从最基础的字符级词表开始，逐步地将出现频率最高的字符对（pair）合并，形成越来越长的子词序列。）

2018年由Google改进了子词分割方法，经常可以在分词之前去除对于特定语言的预处理需求。此外，该论文提出了一种称为子词正则化的新颖的正则化技术，该技术通过在训练期间在分词过程中引入一些随机性来提高精度和稳健性：例如，New England可能被拆分为New和England，也可能拆分为New、Eng和land，还可能拆分为NewEngland（只有一个词元）。Google的SentencePiece项目 (https://github.com/google/sentencepiece) 提供了一个开源实现

TensorFlow Text库还实现了包括WordPiece（BPE的变体）在内的各种分词策略。最后，HuggingFace的Tokenizer库实现了各种速度极快的分词器。然而，对于英文IMDb任务来说，使用空格作为词元边界应该足够了。让我们继续创建一个TextVectorization层并对其进行调整使其适应于训练集。我们将词汇表限制为1000个词元，使其包括最常见的998个单词以及一个填充词元和一个未知单词词元，因为很少见的单词不太可能对此任务有重要作用，而且限制词汇表大小可以减少模型需要学习的参数数量

In [64]:
vocab_size = 1000
text_vec_layer = tf.keras.layers.TextVectorization(max_tokens=vocab_size)
text_vec_layer.adapt(train_set.map(lambda reviews, labels: reviews))

In [57]:
embed_size = 128
tf.random.set_seed(42)
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 288s 405ms/step - loss: 0.6935 - accuracy: 0.5022 - val_loss: 0.6931 - val_accuracy: 0.5020
Epoch 2/2
704/704 [==============================] - 288s 409ms/step - loss: 0.6928 - accuracy: 0.5052 - val_loss: 0.6936 - val_accuracy: 0.4996


In [68]:
text_vec_layer([["hello"],
                ["hello world sleep bad awful good great"]])

<tf.Tensor: shape=(2, 7), dtype=int64, numpy=
array([[  1,   0,   0,   0,   0,   0,   0],
       [  1, 189,   1,  84, 376,  50,  86]], dtype=int64)>

b  第一层是刚刚准备好的TextVectorization层，接着是一个Embedding层，它将单词ID转换为嵌入。在嵌入矩阵中，词汇表中的每个词元对应一行(vocab_size)，每个嵌入维度对应一列（此示例使用128个维度，但这是可以调整的超参数）。接下来，我们使用GRU层和具有单个神经元和sigmoid激活函数的Dense层，因为这是一个二元分类任务：模型的输出将是表示评论对电影表达正面情感的概率的估计值。然后，我们编译模型，并在之前准备的数据集上训练几个轮次（也可以训练更长时间以获得更好的结果）。

不幸的是，如果运行此代码，你通常会发现模型根本学不到任何东西：精度保持在50%左右，不比随机概率更好。为什么会这样？因为这些评论的长度不同，当TextVectorization层将其转换为词元ID序列时，它使用填充词元（ID为0）填充较短的序列，以使它们与批次中的最长序列一样长。因此，大多数序列以许多填充词元结尾——通常有几十甚至几百个填充词元。即使我们正在使用比SimpleRNN层好的GRU层，但其短期记忆仍然不好，所以当它经过许多填充词元时，最后通常会忘记那条评论讨论的内容！一种解决方案是将相等长度的句子批次输入模型（这也会加快训练速度）​。另一种解决方案是让RNN忽略填充词元，这可以使用掩码完成。

### 掩码

使用Keras让模型忽略填充词元很容易：只需在创建Embedding层时添加mask_zero=True。这意味着所有下游层都将忽略填充词元（其ID为0）。如果重新对上一个模型训练几个轮次，会发现验证精度很快达到80%。

这个工作的方式是Embedding层创建了一个等于tf.math.not_equal(inputs，0)的掩码张量：它是一个布尔张量，与输入具有相同的形状，如果词元ID为0，则元素值为False，否则为True。然后，该掩码张量自动被模型传播到下一层。如果该层的call()方法具有mask参数，则自动接收掩码。这使得该层可以忽略适当的时间步。每一层可能会以不同的方式处理掩码，但总的来说，它们只是忽略掩码为False的时间步。例如，当循环层遇到掩码时间步时，它只需复制上一时间步的输出。



In [69]:
model = tf.keras.Sequential([
    text_vec_layer,
    tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True),   # input != 0
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 367s 516ms/step - loss: 0.5251 - accuracy: 0.7392 - val_loss: 0.4177 - val_accuracy: 0.8196
Epoch 2/2
704/704 [==============================] - 357s 508ms/step - loss: 0.3556 - accuracy: 0.8496 - val_loss: 0.3537 - val_accuracy: 0.8500


如果层的supports_masking属性为True，则自动将掩码传播到下一层。只要层具有supports_masking=True属性，就以这种方式继续传播。例如，当return_sequences=True时，循环层的supports_masking属性为True，但当return_sequences=False时，它为False，因为在这种情况下不再需要掩码。因此，如果模型有若干return_sequences=True的循环层，它们后面跟着一个return_sequences=False的循环层，那么掩码将自动传播到最后一个循环层：该层将根据掩码来忽略应被屏蔽的时间步，但不会进一步传播掩码。同样，如果在创建情感分析模型中的Embedding层时设置了mask_zero=True，那么GRU层将自动接收并使用掩码，但不会进一步传播它，因为return_sequences未设置为True。

一些层在将掩码传递给下一层之前需要更新它：它们通过实现compute_mask()方法来完成这种更新，该方法需要两个参数：输入和前一个掩码。该方法将计算更新的掩码并将其返回。compute_mask()方法的默认实现只是返回前一个掩码而已。

如果掩码一直传播到输出，则它也会应用于损失，因此掩码时间步不会对损失产生贡献（它们的损失为0）。这假定模型输出序列，而在我们的情感分析模型中并非如此。

许多Keras层都支持掩码，例如SimpleRNN、GRU、LSTM、Bidirectional、Dense、TimeDistributed、Add等（它们都在tf.keras.layers包中）。但是，卷积层（包括Conv1D）不支持掩码

如果想要实现自己的支持掩码的自定义层，应向call()方法添加一个mask参数，并使该方法使用该掩码。此外，如果要将掩码传播到下一层，则应在构造函数中设置self.supports_masking=True。如果必须在传播之前更新掩码，则必须实现compute_mask()方法。

如果模型不以Embedding层开头，则可以使用tf.keras.layers.Masking层：默认情况下，它将mask设置为tf.math.reduce_any(tf.math.not_equal(X，0)，axis=-1)，这意味着在后续层中将忽略最后一个维度全为零的时间步。

掩码层和自动掩码传播最适用于简单模型。对于更复杂的模型，例如需要将Conv1D层与循环层混合的情况，它并不总是有效。在这种情况下，你需要显式计算掩码并将其传递到适当的层，方法是使用函数式API或子类化API。例如，以下模型与先前的模型相同，只是使用函数式API手动处理掩码。它还增加了一些dropout，因为先前的模型稍微过拟合了

In [7]:
inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
token_ids = text_vec_layer(inputs)
mask = tf.math.not_equal(token_ids, 0)
Z = tf.keras.layers.Embedding(vocab_size, embed_size)(token_ids)
Z = tf.keras.layers.GRU(128, dropout=0.2)(Z, mask=mask)
outputs = tf.keras.layers.Dense(1, activation="sigmoid")(Z)
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])

model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2)

Epoch 1/2
704/704 [==============================] - 344s 484ms/step - loss: 0.5111 - accuracy: 0.7524 - val_loss: 0.3779 - val_accuracy: 0.8372
Epoch 2/2
704/704 [==============================] - 366s 520ms/step - loss: 0.3527 - accuracy: 0.8516 - val_loss: 0.3324 - val_accuracy: 0.8632


最后一种掩码方法是使用不规则张量作为模型输入。在实践中，只需要在创建TextVectorization层时将ragged设置为True，以使输入序列表示为不规则张量：

Keras的循环层本身就支持不规则张量，所以我们无须进行其他操作：只需在模型中使用TextVectorization层即可。无须传递mask_zero=True或显式处理掩码——这一切都已由Keras实现。

无论你喜欢哪种掩码方法，在训练此模型几个轮次之后，它将变得非常擅长判断评论是否正面。如果使用tf.keras.callbacks.TensorBoard()回调，则可以在TensorBoard中可视化嵌入：因为它们正在被学习，所以我们可以看到awesome和amazing等词汇逐渐聚集在嵌入空间的一侧，而awful和terrible等词汇则聚集在另一侧，这很神奇。有些单词并不像我们预期的那样是正面的（至少对于这个模型来说），比如good这个词，这可能是因为许多负面评论包含not good短语。

In [71]:
text_vec_layer_ragged = tf.keras.layers.TextVectorization(max_tokens=vocab_size, ragged=True)

text_vec_layer_ragged.adapt(train_set.map(lambda reviews, labels: reviews))
text_vec_layer_ragged(["Great movie!", "This is DiCaprio's best role."])

text_vec_layer(["Great movie!", "This is DiCaprio's best role."]) # 对比区别

<tf.Tensor: shape=(2, 5), dtype=int64, numpy=
array([[ 86,  18,   0,   0,   0],
       [ 11,   7,   1, 116, 217]], dtype=int64)>

In [6]:
from pathlib import Path
from time import strftime

# 根据当前日期和时间生成日志子目录的路径，以便每次运行时都不同：
def get_run_logdir(root_logdir="my_logs"):
    return Path(root_logdir) / strftime("run_%Y_%m_%d_%H_%M_%S")

run_logdir = get_run_logdir()
run_logdir

WindowsPath('my_logs/run_2026_01_25_15_19_50')

In [7]:
# 确保日志目录存在（包括所有上级目录）
run_logdir.mkdir(parents=True, exist_ok=True)

vocab = text_vec_layer_ragged.get_vocabulary()
vocab_file = run_logdir / "metadata.tsv"

# 写入 metadata.tsv（每行一个词）
with open(vocab_file, "w", encoding="utf-8") as f:
    for word in vocab:
        f.write(f"{word}\n")

tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=run_logdir,
    embeddings_freq=1,  # 每个 epoch 保存一次嵌入
    embeddings_metadata=str(vocab_file)  # 指向 metadata 文件
)

In [8]:
model = tf.keras.Sequential([
    text_vec_layer_ragged,
    tf.keras.layers.Embedding(vocab_size, embed_size),
    tf.keras.layers.GRU(128),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
history = model.fit(train_set, validation_data=valid_set, epochs=2, callbacks=[tensorboard_cb])

# %tensorboard --logdir=./my_logs

Epoch 1/2
704/704 [==============================] - 341s 480ms/step - loss: 0.5062 - accuracy: 0.7480 - val_loss: 0.3699 - val_accuracy: 0.8468
Epoch 2/2
704/704 [==============================] - 348s 495ms/step - loss: 0.3533 - accuracy: 0.8522 - val_loss: 0.3209 - val_accuracy: 0.8672


## 重用预训练的嵌入和语言模型

模型只通过25000条电影评论就能够学到有用的词嵌入，如果用数十亿条评论来训练模型，那么词嵌入的质量将会达到何等高度！可惜，没有这么多的评论数据。

这些词嵌入是基于其他任务训练出来的，它们也可能在情感分析中发挥作用：因为awesome和amazing这类词汇具有相似的含义，所以它们很可能在嵌入空间中聚集在一起，即使是用于句子下一个单词的预测这类任务。如果所有正面单词和所有负面单词都聚集成簇，那么对于情感分析是很有帮助的。所以，可以直接下载并使用已经训练好的词嵌入，比如Google的Word2vec词嵌入、斯坦福大学的GloVe词嵌入或Facebook的FastText词嵌入。

使用预训练的词嵌入在近年来非常流行，但这种方法也有限制。特别是，一个词无论上下文如何，只有一个表示。例如，单词right在“left and right”和“right and wrong”中均以使用预训练的词嵌入在近年来非常流行，即使它代表两种完全不同的含义。为了解决这个限制问题，2018年引入ELMo（Embeddingfrom Language Models，来自语言模型的嵌入）：这些是从深度双向语言模型的内部状态中学习到的语境化词嵌入。与其将预训练嵌入的一部分用在模型中，不如重复使用预训练语言模型的一部分。

大致在同一时间，ULMFiT（Univesal LanguageModel Fine-Tuning，通用语言模型微调）论文证明了无监督预训练对NLP任务的有效性：作者使用自监督学习（即从数据自动生成标签）在庞大的文本语料库上训练了一个LSTM语言模型，然后针对各种任务对其进行微调。他们的模型在6个文本分类任务中表现出色，大多数情况下错误率降低了18%～24%。此外，作者展示了仅使用100个标记样本微调过的预训练模型的性能可以达到从头开始使用1万个样本训练的模型的水平。在ULMFiT论文之前，仅在计算机视觉领域中采用预训练模型才是常规操作；在NLP领域，预训练仅限于词嵌入。这篇论文标志着NLP新时代的开始：现在，重用预训练语言模型已经成为常态。


In [ ]:
"""
加载 Google研究员团队介绍的通用句子编码器构建

这个模型很大——接近1 GB，因此下载可能需要一些时间。默认情况下，TensorFlow Hub模块保存在临时目录中，并且每次运行程序都会重新下载它们。为了避免这种情况，必须将TFHUB_CACHE_DIR环境变量设置为所选择的目录，然后模块将被保存在那里，仅下载一次。
"""
import os
import tensorflow_hub as hub

os.environ["TFHUB_CACHE_DIR"] = "my_tfhub_cache"
model = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                   trainable=True, dtype=tf.string, input_shape=[]),  # 预训练模型可微调
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.compile(loss="binary_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit(train_set, validation_data=valid_set, epochs=10)

## 用于机器翻译的编码器--解码器神经网络

到目前为止，我们已经使用char-RNN执行了文本生成任务，并使用基于可训练词嵌入的单词级RNN模型以及来自TensorFlow Hub的强大预训练语言模型进行了情感分析。在下一节中，将探讨另一个重要的NLP任务：神经机器翻译(NMT)。

简而言之，架构如下：英语句子作为输入被馈入编码器，解码器输出西班牙语翻译。请注意，在训练期间，西班牙语翻译也被用作解码器的输入，但被向后移动了一步。换句话说，在训练期间，解码器被给予它应该在上一个步骤输出的单词作为输入，而不管它实际上输出了什么。这被称为“教师强制”(teacher forcing)——一种显著加速训练并改善模型性能的技术。对于第一个单词，解码器被给予序列起始(Start-of-Sequence，SOS)词元，解码器预计以序列结束(End-of-Sequence，EOS)词元结束句子。每个单词最初由其ID（例如soccer的ID为854）表示。接下来，Embedding层返回词嵌入。这些词嵌入被馈送到编码器和解码器。

每个单词最初由其ID（例如soccer的ID为854）表示。接下来，Embedding层返回词嵌入。这些词嵌入被馈送到编码器和解码器。

在每个步骤中，解码器会为输出词汇表（西班牙语）中的每个单词输出一个评分，然后softmax激活函数会将这些评分转换为概率。例如，在第一步中，Me这个单词的概率可能是7%，Yo的概率可能是1%，以此类推。输出是具有最高概率的单词。这非常类似于常规分类任务，确实可以使用"sparse_categorical_crossentropy"损失训练模型，就像我们在char-RNN模型中所做的那样。

![简单的机器翻译模型](./images/RNN/p11.png)

注意，在推理时（训练之后），不会把目标句子馈送给解码器。相反，你需要馈入它在上一步输出的单词。（需要一个嵌入查找，图中并未显示）

![推理时，将解码器在上一个时间步输出的单词重新作为输入馈送给它](./images/RNN/p12.png)

In [21]:
import tensorflow as tf
from pathlib import Path

In [22]:
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"

path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets, extract=True")


In [23]:
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text(encoding="utf-8")  # 磁盘里解压缩后执行

每行包含一个英语句子和相应的西班牙语翻译，它们由制表符分隔。我们首先将删除TextVectorization层不处理的西班牙字符¡和¿，然后解析句子对并打乱它们。最后，我们将它们分成两个单独的列表，每种语言一个

In [24]:
import numpy as np

text = text.replace("\u00A1", "").replace("\u00BF", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)

In [25]:
for i in range(3):
    print(sentences_en[i], "=>", sentences_es[i])

Tom looks sick. => Tom parece enfermo.
I like none of them. => No me gusta ninguno de ellos.
I don't blame you. => No te culpo.


In [26]:
# 使用文本向量化层
vocab_size = 1000
max_length = 50

text_vec_layer_en = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)
text_vec_layer_es = tf.keras.layers.TextVectorization(vocab_size, output_sequence_length=max_length)

text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

- 将词汇表大小限制为1000，这非常小。那是因为训练集不是很大，而且使用小值会加快训练速度。先进的翻译模型通常使用更大的词汇表（例如大小为30000）、更大的训练集（千兆字节）和更大的模型（数百甚至数千兆字节)

- 由于数据集中的句子最多有50个单词，因此我们将output_sequence_length设置为50：这样输入序列将自动用零填充，直到它们都达到50个词元长。如果训练集中有任何句子超过50个词元，它将被裁剪为50个词元。

- 对于西班牙语文本，我们在调整TextVectorization层时向每个句子添加startofseq和endofseq：我们将使用这些词作为SOS和EOS词元。你可以使用任何其他词，只要它们不是真正的西班牙语词即可。

In [27]:
# 检查一下两个词汇表中的前10个词元。它们从填充词元、未知词元、SOS词元和EOS词元（仅在西班牙语词汇表中有）开始，然后是实际单词（按频率降序排列

text_vec_layer_en.get_vocabulary()[:10]
text_vec_layer_es.get_vocabulary()[:10]

['', '[UNK]', 'startofseq', 'endofseq', 'de', 'que', 'a', 'no', 'tom', 'la']

In [28]:
# 创建训练集和验证集（如果需要，也可以创建测试集）。将使用前100000个句子对进行训练，用其余的进行验证。
# 解码器的输入是西班牙语句子加上SOS词元前缀。目标值是西班牙语句子加上EOS后缀：

X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

In [29]:
# 构建模型，模型不是顺序的，需要两个文本输入（一个用于编码器，一个用于解码器）

encoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=[], dtype=tf.string)


# 句子分词编码后 接嵌入，设置mask_zero=True确保自动处理掩码
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size, mask_zero=True)

encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

# 注意：当语言共享许多词汇时，对编码器和解码器使用相同的嵌入层可能会获得更好的性能。

In [30]:
# 创建编码器并将嵌入传递
encoder = tf.keras.layers.LSTM(512, return_state=True)
encoder_outputs, *encoder_state = encoder(encoder_embeddings)



只使用了一个LSTM层，但也可以堆叠多个。

设置return_state=True，以获取对层的最终状态的引用。由于我们使用了LSTM层，因此实际上有两种状态：短期状态和长期状态。该层将分别返回这些状态，因此我们必须编写*encoder_state以将两个状态在列表中分组

现在可以使用这个（双重）状态作为解码器的初始状态：

In [31]:
decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

# 通过具有softmax激活函数的Dense层传递解码器的输出以获取每个步骤的单词概率
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)

当输出词汇表很大时，输出每个可能单词的概率可能会非常慢。如果目标词汇表包含50000个而不是1000个西班牙语单词，那么解码器将输出50000维向量，并且在如此大的向量上应用softmax函数的计算量会非常大。为避免这种情况，一种解决方案是仅查看模型输出的正确单词和错误单词的随机样本的logit，然后仅基于这些logit计算损失的近似值。这种采样softmax技术于2015年引入。在TensorFlow中，这可以通过在训练期间使用tf.nn.sampled_softmax_loss()函数，并在推理时使用普通的softmax函数（采样softmax不能在推理时使用，因为它需要知道目标值）来实现。

总结：采样 softmax（sampled softmax）是一种“训练时近似计算 loss、推理时仍用完整 softmax”的加速技巧，用来解决词表非常大（比如 5 万词）时，softmax 计算太慢的问题。计算损失时，不用所有的类别概率参与计算，只要采样部分类别（但正确类别一定得有）计算交叉熵损失；推理的时候不能去采样softmax是因为推理是要预测目标了，得从这50000个概率里采样。

另一件可以加快训练速度的事情（与采样softmax兼容）是将输出层的权重与解码器嵌入矩阵的转置绑定。这显著减少了模型参数的数量，从而加快了训练速度，有时还可以提高模型的精度，尤其是在没有大量训练数据的情况下。嵌入矩阵相当于独热编码后跟一个没有偏置项的线性层，也没有将独热向量映射到嵌入空间的激活函数。输出层则做相反的事情。因此，如果模型可以找到转置接近其逆的嵌入矩阵（这样的矩阵称为正交矩阵），则无须为输出层学习一组单独的权重。（embedding的行空间性质很好，不扭曲）

另一层理解方式：在权重绑定中，输出层不再学习独立的分类权重，而是使用词嵌入矩阵的转置，使得输出 logit 等价于解码器隐藏状态与各词嵌入向量的点积，从而基于语义相似度对词进行打分。

In [32]:
# 创建模型，编译+训练
# 警告：用CPU要跑 好几个小时
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10, validation_data=((X_valid, X_valid_dec), Y_valid))

Epoch 1/10
3125/3125 [==============================] - 1593s 508ms/step - loss: 2.9254 - accuracy: 0.4257 - val_loss: 2.1628 - val_accuracy: 0.5257
Epoch 2/10
3125/3125 [==============================] - 1629s 521ms/step - loss: 1.8465 - accuracy: 0.5748 - val_loss: 1.6491 - val_accuracy: 0.6117
Epoch 3/10
3125/3125 [==============================] - 1652s 529ms/step - loss: 1.4365 - accuracy: 0.6481 - val_loss: 1.4328 - val_accuracy: 0.6529
Epoch 4/10
3125/3125 [==============================] - 1641s 525ms/step - loss: 1.2035 - accuracy: 0.6939 - val_loss: 1.3310 - val_accuracy: 0.6742
Epoch 5/10
3125/3125 [==============================] - 1563s 500ms/step - loss: 1.0404 - accuracy: 0.7276 - val_loss: 1.2903 - val_accuracy: 0.6822
Epoch 6/10
3125/3125 [==============================] - 1500s 480ms/step - loss: 0.9094 - accuracy: 0.7552 - val_loss: 1.2827 - val_accuracy: 0.6865
Epoch 7/10
3125/3125 [==============================] - 1505s 482ms/step - loss: 0.7951 - accuracy: 0.7804

训练后，我们可以使用该模型将新的英语句子翻译成西班牙语。但这并不像调用model.predict()那样简单，因为解码器希望将前一时间步预测的单词作为输入。实现此目的的一种方法是编写一个自定义记忆单元来跟踪先前的输出并在下一个时间步将其馈送到编码器。然而，为了简单起见，可以多次调用模型，在每一轮预测一个额外的单词。为此编写一些实用函数：

In [33]:
def translate(sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = np.array([sentence_en])  # 编码器输入
        X_dec = np.array(["startofseq " + translation]) # 解码器输入
        y_proba = model.predict((X, X_dec))[0, word_idx] # 最后一个token的概率分布
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()


In [35]:
print(translate("I like soccer"))
print(translate("I like soccer and also going to the beach"))

1/1 [==============================] - 0s 30ms/step
me gusta el fútbol
1/1 [==============================] - 0s 25ms/step
me gusta el fútbol y al [UNK] al tenis


目前的模型适用于翻译短语，但是在处理较长的句子时很吃力，一种改进方法是增加训练集大小，并在编码器和解码器中添加更多的LSTM层。但这只能在一定程度上有所帮助，因此看看更复杂的技术，从双向循环层开始。


### 双向RNN

在每个时间步，常规循环层在生成输出之前只查看过去和现在的输入。换句话说，这是因果关系，这意味着它无法看到未来。这种类型的RNN在预测时间序列时或在序列到序列(seq2seq)模型的解码器中很有意义。但是对于像文本分类这样的任务，或者在seq2seq模型的编码器中，通常最好在编码给定单词之前先看一下下一个单词。

例如，考虑短语the right arm、the right person和the right to criticize：要正确编码right一词，需要查看下一个词。一种解决方案是在相同的输入上运行两个循环层（一个从左到右读取单词，另一个从右到左读取它们），然后在每个时间步组合它们的输出，通常是将它们连接起来。这就是双向循环层所做的

要在Keras中实现双向循环层，只需将循环层包装在tf.keras.layers.Bidirectional层中。例如，以下Bidirectional层可用作我们翻译模型中的编码器：

![双向循环层](./images/RNN/p8.png)

In [ ]:
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_state=True))

Bidirectional层将创建LSTM层的副本（但方向相反），并且将同时运行它们并连接它们的输出。因此，尽管LSTM层具有256个单元，但Bidirectional层每个时间步将输出512个值。

这个层现在会返回4个状态而不是2个：前向LSTM层的最终短期状态和长期状态，以及后向LSTM层的最终短期状态和长期状态。我们不能直接使用此4重状态作为解码器的LSTM层的初始状态，因为它只期望2个状态（短期状态和长期状态）。我们不能使解码器变成双向的，因为它必须保持因果关系，否则它会在训练期间作弊，而且它将无法工作。相反，我们可以将两个短期状态和两个长期状态分别连接起来：

In [ ]:
encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.concat(encoder_state[::2], axis=-1),  # short-term (0 & 2)
                 tf.concat(encoder_state[1::2], axis=-1)]  # long-term (1 & 3)

decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(decoder_outputs)
model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                       outputs=[Y_proba])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
              metrics=["accuracy"])
model.fit((X_train, X_train_dec), Y_train, epochs=10,
          validation_data=((X_valid, X_valid_dec), Y_valid))

In [ ]:
translate("I like soccer")
translate("I like soccer and also going to the beach")

### 束搜索(Beam Search)

nn贪心算法每次只求出当前句子概率最大的下一个单词。这种算法太容易遗漏更优的输出了。而如果真的求出最优的句子，即求出 argmax_y P(y|x)，需要遍历所有可能的 y。假如每个 y 都有 N 个选择，句子长度为 T_y，则搜索算法的复杂度是 O(N^T_y)。这个指数增长的复杂度是不能接受的。

Beam Search 是这样一种折中的启发式搜索算法。它不能保证找出最优解，却能比贪心算法找出更多更优的解。

Beam Search 的核心思想可以用一句话概括：相比于只维护一个概率最优句子的贪心算法，Beam Search 每次维护 B 个概率最优的句子。

还是拿开头那句话的翻译为例，并假设 B = 3，词汇表大小为 10000。生成第一个单词时，概率最高的三个单词可能是 in, Jane, September。生成第二个单词时，我们要遍历所有 3×10000 种两个单词组合的可能。最终，我们可能发现 September Jane is、Jane is、Jane visits 这三个句子的概率最高。依次类推，我们继续遍历下去，直到生成句子里的所有单词。

![宽度为3的束搜索](./images/RNN/p13.png)

以下代码是非常基础的 Beam Search 实现。尽量让代码易读、易理解，但它在速度方面显然没有做任何优化。

该函数首先使用模型来找出作为翻译开头的前 k 个单词（其中 k 是 beam width，束宽）。对于这 k 个候选翻译中的每一个，函数都会计算在当前翻译后面可能接上的所有单词的条件概率。然后，将这些扩展后的翻译及其对应的概率加入候选列表中。

当我们遍历完所有前 k 个翻译，并考虑了所有可以用于扩展它们的单词之后，只保留概率最高的 k 个候选翻译。这个过程会不断重复，直到所有候选翻译都以 EOS（句子结束）标记结尾。最后，返回概率最高的那个翻译（并去除其末尾的 EOS 标记）。

注意：如果 p(S) 表示句子 S 的概率，p(W|S) 表示在翻译以 S 开头的条件下生成单词 W 的条件概率，那么新句子 S′ = concat(S, W) 的概率为：

p(S′) = p(S) × p(W|S)

随着不断加入新的单词，句子的整体概率会变得越来越小。为了避免概率值过小而导致的浮点数精度问题，函数并不是直接保存概率，而是保存对数概率。回忆一下：

log(a × b) = log(a) + log(b)

因此：

log(p(S′)) = log(p(S)) + log(p(W|S))

In [ ]:
# 额外代码 —— 一个非常基础的 Beam Search 实现

def beam_search(sentence_en, beam_width, verbose=False):
    X = np.array([sentence_en])  # 编码器输入
    X_dec = np.array(["startofseq"])  # 解码器初始输入
    y_proba = model.predict((X, X_dec))[0, 0]  # 第一个 token 的概率分布

    top_k = tf.math.top_k(y_proba, k=beam_width)
    top_translations = [  # 保存当前最优的 beam_width 个翻译：(对数概率, 翻译文本)
        (np.log(word_proba), text_vec_layer_es.get_vocabulary()[word_id])
        for word_proba, word_id in zip(top_k.values, top_k.indices)
    ]

    # 额外代码 —— 在 verbose 模式下显示概率最高的第一个单词
    if verbose:
        print("Top first words:", top_translations)

    for idx in range(1, max_length):
        candidates = []
        for log_proba, translation in top_translations:
            # 如果当前翻译已经以 endofseq 结尾，说明句子已生成完成
            if translation.endswith("endofseq"):
                candidates.append((log_proba, translation))
                continue  # 翻译已完成，不再继续扩展该句子

            X = np.array([sentence_en])  # 编码器输入
            X_dec = np.array(["startofseq " + translation])  # 当前解码器输入
            y_proba = model.predict((X, X_dec))[0, idx]  # 当前时间步的概率分布

            # 尝试将词表中的每个单词接到当前翻译后面
            for word_id, word_proba in enumerate(y_proba):
                word = text_vec_layer_es.get_vocabulary()[word_id]
                candidates.append(
                    (log_proba + np.log(word_proba),
                     f"{translation} {word}")
                )

        # 按对数概率排序，只保留概率最高的 beam_width 个候选翻译
        top_translations = sorted(candidates, reverse=True)[:beam_width]

        # 额外代码 —— 在 verbose 模式下显示当前最优的翻译结果
        if verbose:
            print("Top translations so far:", top_translations)

        # 如果所有候选翻译都已经生成了 endofseq，则结束搜索
        if all([tr.endswith("endofseq") for _, tr in top_translations]):
            return top_translations[0][1].replace("endofseq", "").strip()


In [ ]:
sentence_en = "I love cats and dogs"
translate(sentence_en)

# beam_search(sentence_en, beam_width=3, verbose=True)

### 注意力机制

刚刚学习的这种“编码器-解码器”架构的RNN确实能在机器翻译上取得不错的效果。但是，这种架构存在一定的限制：模型的编码（输入）和解码（输出）这两步都是一步完成的，模型一次性输入所有的句子，一次性输出所有的句子。这种做法在句子较短的时候还比较可行，但输入句子较长时，模型就“记不住”之前的信息了。而我们这一节学习的注意力模型能够很好地处理任意长度的句子。

Bahdanau等人在2014年发表的具有里程碑意义的论文中引入了一种技术，使解码器能够在每个时间步专注于适当的词（由编码器编码）。例如，在需要输出单词fútbol的时间步，解码器会将注意力集中在单词soccer上。这意味着从输入单词到其翻译的路径现在要短得多，因此RNN的短期记忆限制的影响要小得多。注意力机制彻底改变了神经机器翻译（以及一般的深度学习），显著改进了现有技术，特别是对于长句子（例如，超过30个单词的句子）。

下图显示了带有注意力机制的编码器—解码器模型。左侧是编码器和解码器。我们现在将编码器的所有输出都发送到解码器，而不仅仅是在每个步骤中将编码器的最终隐藏状态和先前的目标单词发送给解码器（虽然这仍然需要完成，但图中并未显示）。由于解码器无法一次处理所有这些编码器输出，因此它们需要聚合：在每个时间步中，解码器的记忆单元计算编码器所有输出的加权和。这可以确定它在此步骤将关注哪些单词。权重α(t，i)是第i个编码器输出在第t个解码器时间步的权重。例如，如果权重α(3，2)比权重α(3，0)和α(3，1)大得多，则解码器将在这个时间步中更加关注单词#2（即soccer）的编码器输出，而不是其他两个输出。解码器的其余部分工作方式与之前相同：在每个时间步中，记忆单元接收刚刚讨论的输入以及上一个时间步的隐藏状态，最后（尽管它没有在图中表示）接收上个时间步的目标单词（在推理时，为上一个时间步的输出）。

![注意力+编码器解码器的翻译模型](./images/RNN/p14.png)

但是，这些α(t，i)权重是从哪里来的呢？它们是由一个小型神经网络［称为对齐模型（或注意力层）］生成的，该模型与其余的编码器—解码器模型共同训练。这个对齐模型如图16-7的右边所示。它从由一个神经元组成的Dense层开始，该层处理编码器的每个输出以及解码器的前一个隐藏状态（例如h(2)）​。该层为每个编码器输出（例如e(3，2)）输出一个分数（或能量）：此分数度量每个输出与解码器的前一个隐藏状态对齐的程度。例如，在图16-7中，模型已经输出“me gusta el”​（意思是“Ilike”​）​，所以它现在期望一个名词：单词soccer是与当前状态对齐程度最高的，因此它获得高分。最后，所有分数都经过softmax层，以获得每个编码器输出的最终权重（例如α(3，2)）。给定解码器时间步的所有权重加起来为1。这种特定的注意力机制称为“Bahdanau注意力",由于它将编码器输出与解码器的前一个隐藏状态,接起来，因此有时被称为“连接注意力”或“加性注意力”。